In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import neologdn
import os
import urllib.request
from matplotlib.font_manager import FontProperties
import MeCab
from __future__ import unicode_literals
import re
import unicodedata
import sagemaker
from sagemaker import get_execution_role
import boto3
import json
import pandas as pd
import warnings
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import codecs
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

#chaneg dummies to matrix
from scipy.sparse import csr_matrix,coo_matrix

import multiprocessing

from multiprocessing import Pool

import pickle

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout, Input, concatenate
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Model
import os
import tarfile
import numpy as np

import string

In [ ]:
#tokenize japanase words
def wakati_by_mecab(text):
    tagger = MeCab.Tagger('-Ochasen -d /usr/lib64/mecab/dic/mecab-ipadic-neologd')
    tagger.parse('') 
    node = tagger.parseToNode(text)
    word_list = []
    while node:
        pos = node.feature.split(",")[0]
        #if pos in ["名詞","動詞","形容詞","記号"]:   # 対象とする品詞
        if pos in ["名詞","動詞","形容詞"]:   # 対象とする品詞
            word = node.surface
            word_list.append(word)
        node = node.next
    return " ".join(word_list)

In [ ]:
def remove_chars(row):
    cust_list=['\d+','■','\n','・','◇','①','②','③','④','【】','】','【']
    del_list = string.ascii_letters + string.punctuation
    for i in del_list :
        row = row.str.replace(i,'')
    for i in cust_list :
        row = row.str.replace(i,'')
    return row

In [ ]:
def get_df_from_excel_2018(file_path):
    df = pd.read_excel(file_path, sheetname=None, ignore_index=True)
    df_sheet_all = pd.concat(df.values())
    cols=['Comment_Catregory_”Brochure”','Comment_Catregory_”Compass”','Comment_Catregory_”My AXA”','Comment_Catregory_”Reputation”',
          'Comment_Catregory_”商品/契約内容”','Comment_Catregory_”手続き_（⊆ Compass)”','Comment_Catregory_”担当者”'
         ,'その他' 
         ,'comment' 
         ,'Q4： \n質問3でつけた点数の理由として、良いと思った点'
         ,'Q5：\n（質問3でより高い点数をつけるために）改善が必要だと思う点']
    df_col=df_sheet_all[cols]
    df_col.columns = ['cat_brochure', 'cat_compass', 'cat_myaxa', 'cat_reputaion', 'cat_keiyaku','cat_tetuduki','cat_tanto','cat_other','cmt0','cmt1','cmt2']
    return df_col
    
    

In [ ]:
def get_df_from_excel_2019(file_path):
    df = pd.read_excel(file_path, sheetname=None, ignore_index=True)
    df_sheet_all = pd.concat(df.values())
    cols=['Comment_Catregory_”Brochure”','Comment_Catregory_”Compass”','Comment_Catregory_”My AXA”','Comment_Catregory_”Reputation”',
      'Comment_Catregory_”商品/契約内容”','Comment_Catregory_”手続き_（⊆ Compass)”','Comment_Catregory_”担当者”'
     ,'その他' 
     ,'comment' 
     ,'質問4： NPSでつけた点数の理由として、良いと思った点について次のうちから最も当てはまるものを選んでください。'
     ,'質問5：（質問3でより高い点数をつけていただくために）改善が必要だと思う点について、次のうちから最も当てはまるものを選んでください。'
     ,'質問8： 今回の契約に至った決め手を教えてください。'
     ,'質問9： 担当FA（フィナンシャルプランアドバイザー）について、良い点・改善すべき点等のご意見、激励・応援メッセージ、ライフプランの感想などをご記入ください。（必須）'
     ,'質問5：質問4でつけた評価の理由として、次のなかから最も当てはまるものを選んでください。（必須）'
     ,'質問6：質問4でつけた点数に関して、具体的な感想やコメントをお聞かせください。（任意）']
    df_col=df_sheet_all[cols]
    df_col.columns = ['cat_brochure', 'cat_compass', 'cat_myaxa', 'cat_reputaion', 'cat_keiyaku','cat_tetuduki','cat_tanto','cat_other','cmt0','cmt1','cmt2','cmt3','cmt_4','cmt_5','cmt_6']
    return df_col
    

In [ ]:
def get_df_from_excel_2019_sheet(file_path,sheet_name):
    df_sheet_all = pd.read_excel(file_path, sheetname=sheet_name, ignore_index=True)
    #df_sheet_all = pd.concat(df.values())
    cols=['Comment_Catregory_”Brochure”','Comment_Catregory_”Compass”','Comment_Catregory_”My AXA”','Comment_Catregory_”Reputation”',
      'Comment_Catregory_”商品/契約内容”','Comment_Catregory_”手続き_（⊆ Compass)”','Comment_Catregory_”担当者”'
     ,'その他' 
     ,'comment' 
     ,'質問4： NPSでつけた点数の理由として、良いと思った点について次のうちから最も当てはまるものを選んでください。'
     ,'質問5：（質問3でより高い点数をつけていただくために）改善が必要だと思う点について、次のうちから最も当てはまるものを選んでください。'
     ,'質問8： 今回の契約に至った決め手を教えてください。'
     ,'質問9： 担当FA（フィナンシャルプランアドバイザー）について、良い点・改善すべき点等のご意見、激励・応援メッセージ、ライフプランの感想などをご記入ください。（必須）'
     ,'質問5：質問4でつけた評価の理由として、次のなかから最も当てはまるものを選んでください。（必須）'
     ,'質問6：質問4でつけた点数に関して、具体的な感想やコメントをお聞かせください。（任意）']
    df_col=df_sheet_all[cols]
    df_col.columns = ['cat_brochure', 'cat_compass', 'cat_myaxa', 'cat_reputaion', 'cat_keiyaku','cat_tetuduki','cat_tanto','cat_other','cmt0','cmt1','cmt2','cmt3','cmt_4','cmt_5','cmt_6']
    return df_col

In [ ]:
file_path="/home/ec2-user/SageMaker/Notebooks_For_CX_Usecases/Multiple_Labels_Text_Classfication//raw_data/processed_ML_TXT_CS_data.csv"
df_mutilabel=pd.read_csv(file_path)
df_mutilabel.dropna(inplace=True)

In [ ]:
df_mutilabel.shape

In [ ]:
df_mutilabel.head(5)

In [ ]:
from sklearn.model_selection import train_test_split

#train, test = train_test_split(df_mutilabel, test_size=0.2)

train_texts = df_mutilabel["comment"].values
train_labels = df_mutilabel[['cat_brochure', 'cat_compass', 'cat_myaxa', 'cat_reputaion', 'cat_keiyaku','cat_tetuduki','cat_tanto','cat_other']].values

#test_texts = test["comment"].values
#test_labels = test[['cat_brochure', 'cat_compass', 'cat_myaxa', 'cat_reputaion', 'cat_keiyaku','cat_tetuduki','cat_tanto','cat_other']].values



In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

text=list(df_mutilabel['comment'].values)


top_vocab=3000

tokenizer = Tokenizer(top_vocab)
#list xは上に書いたようにテキストデータが格納されている
tokenizer.fit_on_texts(text)

maxlen_value=50

# 对每一句影评文字转换为数字列表，使用每个词的编号进行编号
x_train_seq = tokenizer.texts_to_sequences(train_texts)
#x_test_seq = tokenizer.texts_to_sequences(test_texts)
x_train = sequence.pad_sequences(x_train_seq, maxlen=maxlen_value)
#x_test = sequence.pad_sequences(x_test_seq, maxlen=maxlen_value)
y_train = np.array(train_labels)
#y_test = np.array(test_labels)
with open('LSTM-GRU-Stack-mutillabel_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import copy
x = list(tokenizer.word_counts.items())
s = sorted(x,key=lambda p:p[1],reverse=True)
word_index = tokenizer.word_index # 不受vocab_size的影响
small_word_index = copy.deepcopy(word_index) # 防止原来的字典也被改变了
print("Removing less freq words from word-index dict...")
for item in s[top_vocab:]:
    small_word_index.pop(item[0])
print("Finished!")
print(len(small_word_index))
print(len(word_index))

In [ ]:
import gensim
#配布されていたfastTextの次元数は300だったので300
EMBEDDING_DIM = 200
#embedding_metrix = np.zeros(top_vocab+1,EMBEDDING_DIM)
embedding_matrix = np.random.uniform(size=(top_vocab+1,EMBEDDING_DIM))
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/home/ec2-user/SageMaker/entitiy_vector_dic/entity_vector/entity_vector.model.bin', binary=True)
for word, i in small_word_index.items():
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        #print("Word: [",word,"] not in wvmodel! Use random embedding instead.")
        pass

In [ ]:
params = [
    {'name': 'Dropout_0',        'type': 'continuous',  'domain': (0.0, 0.5)},
    {'name': 'Dropout_1',        'type': 'continuous',  'domain': (0.0, 0.5)},
    {'name': 'Dropout_2',        'type': 'continuous',  'domain': (0.0, 0.5)},
    {'name': 'Dropout_3',        'type': 'continuous',  'domain': (0.0, 0.5)},
    {'name': 'Dense_LSTM',          'type': 'discrete',    'domain': (64,128,256,512)},
    {'name': 'Dense_GRU',          'type': 'discrete',    'domain': (64,128,256,512)},
    {'name': 'Dense_0',          'type': 'discrete',    'domain': (64,128,256,512)},
    {'name': 'epoch_num',          'type': 'discrete',    'domain': (10,30,60,100)},
    {'name': 'batch_size',          'type': 'discrete',    'domain': (32,64,128,256,512)},
    {'name': 'validation_split', 'type': 'continuous',  'domain': (0.1, 0.3, 0.4)},
    {'name': 'learn_rate',          'type': 'discrete',    'domain': (0.001, 0.01, 0.1, 0.2, 0.3, 0.4)},
]

In [ ]:
def full_multi_label_metric(y_true, y_pred):
    comp = K.equal(y_true, K.round(y_pred))
    return K.cast(K.all(comp, axis=-1), K.floatx())

In [ ]:
import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.layers import Dense, Embedding, Input, concatenate, Flatten, SpatialDropout1D
from keras.layers import CuDNNLSTM,LSTM, Bidirectional, GlobalMaxPool1D, Dropout, CuDNNGRU,GRU, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, LearningRateScheduler, Callback
from keras.optimizers import Adam, Adadelta, SGD, RMSprop, Nadam

def rnn_model(x):
    
    def Param(p_name):
        p_index = [p['name'] for p in params].index(p_name)
        p_type = params[p_index]['type']
    
        if type(x) is np.ndarray:
            if p_type == 'continuous':
                return float(x[:, p_index])
            else:
                return int(x[:, p_index])
        else: # list
            if p_type == 'continuous':
                return float(params[p_index]['domain'])
            else:
                return int(params[p_index]['domain'])
        
    inputs = Input(shape=(maxlen_value,))
    wv_dim=200
    vocab_size=top_vocab
    use_pretrained_wv = True
    if use_pretrained_wv:
        wv = Embedding(vocab_size+1,wv_dim,input_length=maxlen_value,weights=[embedding_matrix],trainable=False)(inputs)
    else:
        wv = Embedding(vocab_size+1,wv_dim,input_length=MAXLEN)(inputs)
        
    layer = SpatialDropout1D(Param('Dropout_0'))(wv)
    layer = Bidirectional(CuDNNLSTM(Param('Dense_LSTM'), return_sequences=True))(layer)
    layer = Dropout(Param('Dropout_1'))(layer)
    layer = Bidirectional(CuDNNGRU(Param('Dense_GRU'), return_sequences=True))(layer)
    layer = Dropout(Param('Dropout_2'))(layer)
    avg_pool = GlobalAveragePooling1D()(layer)
    max_pool = GlobalMaxPooling1D()(layer)
    layer = concatenate([avg_pool, max_pool])
    layer = Dense(Param('Dense_0'), activation="relu")(layer)
    layer = Dense(8, activation="sigmoid")(layer)
    model = Model(inputs=inputs, outputs=layer)
    #model.summary()
    opt = Adam(lr=Param('learn_rate'))
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy',full_multi_label_metric])
    hist= model.fit(x_train,
          y_train,
          batch_size=Param('batch_size'),
          epochs=Param('epoch_num'),
          shuffle=True,
          validation_split=Param('validation_split'),
          callbacks=callbacks,
          verbose=1)
    
    loss = hist.history['val_loss'][-1]
    acc = hist.history['val_acc'][-1]
    full_muti= hist.history['val_full_multi_label_metric'][-1]
    
    global cnt
    print(': Test_loss:', loss, ', Test_acc:', acc, 'full_multi_label_metric', full_muti)
    print('Model: ', model, '\n\n')
    return loss, acc, full_muti,model, hist

In [ ]:
checkpoint_path = "models/ML_TXT_CS_LSTM-GRU-Stack-weights-improvement-{val_full_multi_label_metric:.4f}.hdf5"
checkpoint_dir = os.path.dirname(checkpoint_path)
 
cp_callback = ModelCheckpoint(checkpoint_path,
                              save_best_only=True,
                              verbose=0)
 
callbacks = [
    EarlyStopping(patience=150, monitor='val_full_multi_label_metric',mode='max'),
    cp_callback
]

In [ ]:
def f(x):
    loss, acc, full_muti,model, hist = rnn_model(x)
    return -full_muti

In [ ]:
import GPy
import GPyOpt
opt = GPyOpt.methods.BayesianOptimization(f=f, 
                                          domain=params)

In [ ]:
batch_size = 64
epochs = 50
m = get_model()

m.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,shuffle=True,validation_split=0.2,callbacks=callbacks,verbose=1)


In [ ]:
checkpoint_path = "models/ML_TXT_CS_LSTM-GRU-Stack-weights-improvement-{val_acc:.4f}.hdf5"
checkpoint_dir = os.path.dirname(checkpoint_path)
 
cp_callback = ModelCheckpoint(checkpoint_path,
                              save_best_only=True,
                              verbose=0)
 
callbacks = [
    EarlyStopping(patience=150, monitor='val_acc',mode='max'),
    cp_callback
]

In [ ]:
from keras.optimizers import Adam, Adadelta, SGD, RMSprop, Nadam, Adamax, Adagrad
def return_opt(opt_name):
    if opt_name=='Adam':
        return Adam
    elif opt_name=='Adadelta':
        return Adadelta
    elif opt_name=='SGD':
        return SGD
    elif opt_name=='RMSprop':
        return RMSprop
    elif opt_name=='Nadam':
        return Nadam
    elif opt_name=='Adamax':
        return Adamax
    elif opt_name=='Adagrad':
        return Adagrad

In [ ]:
import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from keras.layers import Dense, Embedding, Input, concatenate, Flatten, SpatialDropout1D
from keras.layers import CuDNNLSTM,LSTM, Bidirectional, GlobalMaxPool1D, Dropout, CuDNNGRU,GRU, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, LearningRateScheduler, Callback
from keras.optimizers import Adam, Adadelta, SGD, RMSprop, Nadam, Adamax, Adagrad
from keras.layers.advanced_activations import PReLU,LeakyReLU

def get_model(activation="relu", optimizer="Adam",learn_rate=0.01,momentum=0,out_dim_lstm=256,out_dim_gru=128,out_dim=64,dropout_rate=0.5, weight_constraint=0):
    inputs = Input(shape=(maxlen_value,))
    wv_dim=200
    vocab_size=top_vocab
    use_pretrained_wv = True
    if use_pretrained_wv:
        wv = Embedding(vocab_size+1,wv_dim,input_length=maxlen_value,weights=[embedding_matrix],trainable=False)(inputs)
    else:
        wv = Embedding(vocab_size+1,wv_dim,input_length=MAXLEN)(inputs)
        
    x = SpatialDropout1D(dropout_rate)(wv)
    x = Bidirectional(CuDNNLSTM(out_dim_lstm, return_sequences=True))(x)
    x = Dropout(dropout_rate)(x)
    x = Bidirectional(CuDNNGRU(out_dim_gru, return_sequences=True))(x)
    x = Dropout(dropout_rate)(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])
    x = Dense(out_dim, activation=activation)(x)
    x = Dense(8, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=x)
    model.summary()
    opt=return_opt(optimizer)
    optimizer_eng = opt(lr=learn_rate, momentum=momentum)
    model.compile(loss='binary_crossentropy', optimizer=optimizer_eng, metrics=['accuracy'])
    return model

In [ ]:
activation = ["relu", "PReLU","LeakyReLU"]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
out_dim = [64, 128, 256, 512]
dropout_rate=[0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
nb_epoch = [10, 25, 40, 60, 100]
batch_size = [10, 32, 64, 128, 256, 512]
param_grid = dict(activation=activation,
                  optimizer=optimizer,
                  learn_rate=learn_rate,
                  momentum=momentum,
                  out_dim=out_dim,
                  dropout_rate=dropout_rate,
                  nb_epoch=nb_epoch,
                  batch_size=batch_size,
                 )

In [ ]:
model = KerasClassifier(build_fn=get_model)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
grid = GridSearchCV(estimator=model,cv=5, param_grid=param_grid, n_jobs=1)

In [ ]:
grid_result = grid.fit(x_train, y_train,validation_split = 0.2,verbose=1,callbacks=callbacks)

In [ ]:
import tensorflow as tf
from keras import backend as K

def full_multi_label_metric(y_true, y_pred):
    comp = K.equal(y_true, K.round(y_pred))
    return K.cast(K.all(comp, axis=-1), K.floatx())

input = tf.keras.layers.Input(shape=(maxlen_value,))
 
x = tf.keras.layers.Embedding(top_vocab+1, EMBEDDING_DIM, weights=[embedding_matrix], trainable=False)(input)

x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True, dropout=0.1,recurrent_dropout=0.1))(x)

x = tf.keras.layers.Conv1D(64, kernel_size=3, padding="valid", kernel_initializer="glorot_uniform")(x)
 
avg_pool = tf.keras.layers.GlobalAveragePooling1D()(x)
max_pool = tf.keras.layers.GlobalMaxPooling1D()(x)
 
x = tf.keras.layers.concatenate([avg_pool, max_pool])
 
preds = tf.keras.layers.Dense(8, activation="sigmoid")(x)
 
model = tf.keras.Model(input, preds)
 
model.summary()
 
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=1e-3), metrics=['accuracy',full_multi_label_metric])

In [ ]:
batch_size = 64
 
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
 
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=0)
 
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=200, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    cp_callback
]
 
model.fit(x_train, y_train, validation_split=0.3, batch_size=batch_size,epochs=25,shuffle=True,callbacks=callbacks, verbose=1)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
latest = tf.train.latest_checkpoint(checkpoint_dir)
 
model.load_weights(latest)
scores = model.evaluate(x_test, y_test)
print('test_loss: %f, accuracy: %f' % (scores[0], scores[1]))

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
 
model.load_weights(latest)
 
predictions = model.predict(x_test)

thrs=np.linspace(0, 1, 100)
sc = np.zeros((len(thrs)))
for i, thr in enumerate(thrs):
    sc[i] = accuracy_score(y_test, (predictions>thr).astype(int))
print("Best threshold {} with score {}".format(thrs[np.argmax(sc)], np.max(sc)))

In [ ]:
latest

In [ ]:
x_test.shape

In [ ]:
model.metrics_names

In [ ]:

latest = tf.train.latest_checkpoint(checkpoint_dir)
 
model.load_weights(latest)
 
predictions = model.predict(x_test)
 
#print(tokenizer.sequences_to_texts(x_test))
print(y_test)
print(predictions)

In [ ]:
def set_threshold(row,threshold_value):
    row = (row > threshold_value)+0;
    return row

In [ ]:
np.mean(predictions,axis=0)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

In [ ]:
#set threshold value
pred=set_threshold(predictions, 0.5)

In [ ]:
print(accuracy_score(y_test, pred, normalize=True))

In [ ]:
i

In [ ]:
thrs=np.linspace(0, 1, 100)
sc = np.zeros((len(thrs)))
for i, thr in enumerate(thrs):
    sc[i] = accuracy_score(y_test, (predictions>thr).astype(int))
print("Best threshold {} with score {}".format(thrs[np.argmax(sc)], np.max(sc)))

In [ ]:
for k in range(pred.shape[1]):
    print("Class", k)
    print(accuracy_score(y_test[:,k], pred[:,k]))

In [ ]:
precision_score(pred,  y_test, average='macro')

In [ ]:
np.mean(pred, axis =0)

In [ ]:
test['predict']=pred.tolist()

In [ ]:
df_5=get_df_from_excel_2019_sheet('comment_2019.xlsx','5月')
cols = ['cmt0','cmt1','cmt2','cmt3','cmt_4','cmt_5','cmt_6']
df_5['comment'] =df_5[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df_5.drop(columns=cols,inplace=True)
df_5['comment']=df_5['comment'].str.replace('nan', '')

In [ ]:
df_5.fillna(0,inplace=True)

for col in ['cat_brochure', 'cat_compass', 'cat_myaxa', 'cat_reputaion', 'cat_keiyaku','cat_tetuduki','cat_tanto','cat_other']:
    df_5[col][(df_5[col]!=0)]=1

for col in ['cat_brochure', 'cat_compass', 'cat_myaxa', 'cat_reputaion', 'cat_keiyaku','cat_tetuduki','cat_tanto','cat_other']:
    df_5[col] = df_5[col].astype('int')

df_5['comment'] = remove_chars(df_5['comment'])
df_5['comment']=df_5['comment'].apply(neologdn.normalize)
df_5['comment']=df_5['comment'].apply(wakati_by_mecab)

In [ ]:
test_texts = df_5["comment"].values
test_labels = df_5[['cat_brochure', 'cat_compass', 'cat_myaxa', 'cat_reputaion', 'cat_keiyaku','cat_tetuduki','cat_tanto','cat_other']].values

x_test_seq = tokenizer.texts_to_sequences(test_texts)
x_test = sequence.pad_sequences(x_test_seq, maxlen=maxlen_value)
y_test = np.array(test_labels)

In [ ]:
scores = model.evaluate(x_test, y_test)
print('test_loss: %f, accuracy: %f' % (scores[0], scores[1]))

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
 
model.load_weights(latest)
 
predictions = model.predict(x_test)

pred=set_threshold(predictions,0.5)
 
df_5['predict']=pred.tolist()

In [ ]:
df_5['predict'].to_csv("predict.csv",index=None)

In [ ]:
import keras
from keras import backend as K

In [ ]:
def threshold_binary_accuracy(y_true, y_pred):
    threshold = 0.80
    if K.backend() == 'tensorflow':
        return K.mean(K.equal(y_true, K.tf.cast(K.lesser(y_pred,threshold), y_true.dtype)))
    else:
        return K.mean(K.equal(y_true, K.lesser(y_pred,threshold)))

In [ ]:
print K.eval(keras.metrics.binary_accuracy(a_true, a_pred))